# Assignment 2

## AI usage 

I generally prefer coding by myself over using AI for help. For this assignment I did, however, find it necessary to use AI to increase my efficiency. I ran into a lot of more techincal problems that would have taken an unreasonable time to solve without AI help.

I used DeepSeek for help on how to connect Cassandra and Spark. I also used DeepSeek for troubleshooting issues I've had with connections and conversion between formats. The finished product is still written by me with mine and AI's solutions combined where needed.

## Log 

The goal of this second assignment is to connect to the database systems Cassandra and MongoDB, using Spark as a tool for data handling. This is first carried out in the Jupyter Notebook and subsequently used in the streamlit app. I will also be connecting to an online API for data collection. 

For this assignment I will try to work more systematic than the last. I will focus on finishing all the elements for the notebook first, and work on the streamlit app afterwards. 

I started by connecting Cassandra and Spark, but ran into problems regarding the sparksession. After troubleshooting for a while and getting some help, we ended up using DeepSeek for help on how to maybe fix it. DeepSeek suggested adding the last three lines of code to force sessionbuilder to use the localhost. 

I have struggled a bit with connecting to Spark and MongoDB, especially lagging a few weeks behind on lectures. I think I have managed to connect properly in the end, though. 

I don't know why my plots are fully black, but I chose not to spend much time on trying to fix it.

Lastly, I once again used mye Jupyter Notebook for testing code snippets before applying them in the app scripts. 

## Links 

- Github: https://github.com/Satheris/IND320_SMAA
- Streamlit app: https://ind320smaa-2eg32uba6uhmrknkwtxzar.streamlit.app/

## Coding

### Imports and system variables

In [ ]:
import numpy as np
import pandas as pd 
import streamlit as st
import pymongo
from cassandra.cluster import Cluster
from pyspark.sql import SparkSession
from pyjstat import pyjstat
import requests
import json
import plotly.express as px
from pyspark import SparkConf, SparkContext

import plotly.io as pio
pio.renderers.default = "notebook+pdf"  # Renderer for Notebook and HTML exports + Renderer for PDF exports

In [2]:
# Set environment variables for PySpark (system and version dependent!) 
# if not already set persistently (e.g., in .bashrc or .bash_profile or Windows environment variables)
import os
# Set the Java home path to the one you are using ((un)comment and edit as needed):
os.environ["JAVA_HOME"] = r"C:\Program Files\Java\jre1.8.0_471"

# If you are using environments in Python, you can set the environment variables like the alternative below.
# The default Python environment is used if the variables are set to "python" (edit if needed):
os.environ["PYSPARK_PYTHON"] = "python" # or similar to "/Users/kristian/miniforge3/envs/tf_M1/bin/python"
os.environ["PYSPARK_DRIVER_PYTHON"] = "python" # or similar to "/Users/kristian/miniforge3/envs/tf_M1/bin/python"

# On Windows you need to specify where the Hadoop drivers are located (uncomment and edit if needed):
os.environ["HADOOP_HOME"] = r"C:\Users\saraa\Documents\winutils\hadoop-3.3.1"

# Set the Hadoop version to the one you are using, e.g., none:
os.environ["PYSPARK_HADOOP_VERSION"] = "without"

### Cassandra and Spark

In [3]:
# Connecting to Cassandra
cluster = Cluster(['localhost'], port=9042)
session = cluster.connect()

In [4]:
# Set up new keyspace
#                                              name of keyspace                        replication strategy           replication factor
session.execute("CREATE KEYSPACE IF NOT EXISTS ind320_keyspace WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 };")

# Create a new table
session.set_keyspace('ind320_keyspace')
session.execute("DROP TABLE IF EXISTS ind320_keyspace.elhub_api;") # Starting from scratch every time
session.execute("CREATE TABLE IF NOT EXISTS elhub_api (ind int PRIMARY KEY, endTime text, lastUpdatedTime text, priceArea text, productionGroup text, quantityKwh float, startTime text);")

In [5]:
spark = SparkSession.builder.appName('SparkCassandraApp').\
    config('spark.jars.packages', 'com.datastax.spark:spark-cassandra-connector_2.12:3.5.1').\
    config('spark.cassandra.connection.host', 'localhost').\
    config('spark.sql.extensions', 'com.datastax.spark.connector.CassandraSparkExtensions').\
    config('spark.sql.catalog.mycatalog', 'com.datastax.spark.connector.datasource.CassandraCatalog').\
    config('spark.cassandra.connection.port', '9042').\
    config('spark.driver.host', 'localhost').\
    config('spark.driver.bindAddress', '127.0.0.1').\
    config('spark.sql.adaptive.enabled', 'true').\
        getOrCreate()

#### Testing that the connection works

In [6]:
# .load() is used to load data from Cassandra table as a Spark DataFrame
spark.read.format("org.apache.spark.sql.cassandra").options(table="my_first_table", keyspace="my_first_keyspace").load().show()

+---+--------+-------+
|ind| company|  model|
+---+--------+-------+
|460|    Ford|Transit|
|459|    Ford| Escort|
|  2|   Tesla|Model 3|
|  3|Polestar|      3|
|  1|   Tesla|Model S|
+---+--------+-------+



In [7]:
# Read CSV file into Spark DataFrame
# planets = spark.read.csv("../data/planets.csv", header=True, inferSchema=True)
# planets.show()

### MongoDB

In [8]:
def init_connection():
    return pymongo.MongoClient(st.secrets["mongo"]["uri"])

client = init_connection()

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


### Elhub API

In [9]:
URL = 'https://api.elhub.no/energy-data/v0/price-areas?dataset=PRODUCTION_PER_GROUP_MBA_HOUR' \
        '&startDate=2021-01-01T00:00:00%2B02:00&endDate=2021-02-01T00:00:00%2B02:00'

payload = { 
    "query": [], 
    "response": { "format": "json-stat2" } }

response = requests.get(URL, json=payload)
data = response.json()

# Writing the data into a file
with open(r'data\api_response.json', 'w', encoding='utf-8') as f:
    json.dump(response.json(), f, indent=2, ensure_ascii=False)
print("Response saved to 'api_response.json'")


# Prints for status
print("\nStatus Code:", response.status_code)
print("Headers:", response.headers.get('content-type'))


# Extract all production records
all_records = []
for area in data['data']:
    records = area['attributes']['productionPerGroupMbaHour']
    for record in records:
        record['priceArea'] = area['attributes']['name']  # Add area name
        all_records.append(record)

df = pd.DataFrame(all_records)
print(f"\nCreated DataFrame with {len(df)} rows")
df.head()

Response saved to 'api_response.json'

Status Code: 200
Headers: application/json; charset=utf-8

Created DataFrame with 17856 rows


,endTime,lastUpdatedTime,priceArea,productionGroup,quantityKwh,startTime
0,2021-01-01T01:00:00+01:00,2024-12-20T10:35:40+01:00,NO1,hydro,2507716.8,2021-01-01T00:00:00+01:00
1,2021-01-01T02:00:00+01:00,2024-12-20T10:35:40+01:00,NO1,hydro,2494728.0,2021-01-01T01:00:00+01:00
2,2021-01-01T03:00:00+01:00,2024-12-20T10:35:40+01:00,NO1,hydro,2486777.5,2021-01-01T02:00:00+01:00
3,2021-01-01T04:00:00+01:00,2024-12-20T10:35:40+01:00,NO1,hydro,2461176.0,2021-01-01T03:00:00+01:00
4,2021-01-01T05:00:00+01:00,2024-12-20T10:35:40+01:00,NO1,hydro,2466969.2,2021-01-01T04:00:00+01:00


Now that I successfully imported 1 month, I need to import for all twelve months. Maximum allowed data range is 1 month, so I need to extract the data in a for-loop. 

In [10]:
monthStart = ['2021-01-01', '2021-02-01', '2021-03-01',
              '2021-04-01', '2021-05-01', '2021-06-01',
              '2021-07-01', '2021-08-01', '2021-09-01',
              '2021-10-01', '2021-11-01', '2021-12-01',
              '2022-01-01']


all_records = []

for i, month in enumerate(monthStart[:12]):
    URL = 'https://api.elhub.no/energy-data/v0/price-areas?dataset=PRODUCTION_PER_GROUP_MBA_HOUR&'\
        f'startDate={month}T00:00:00%2B02:00&endDate={monthStart[i+1]}T00:00:00%2B02:00'

    payload = { 
        "query": [], 
        "response": { "format": "json-stat2" } }

    response = requests.get(URL, json=payload)
    
    # print("\nStatus Code:", response.status_code)

    data = response.json()

    for area in data['data']:
        records = area['attributes']['productionPerGroupMbaHour']
        for record in records:
            record['priceArea'] = area['attributes']['name']
            all_records.append(record)

df = pd.DataFrame(all_records)
df.index.name = 'ind'
df = df.reset_index()

print(f"\nCreated DataFrame with {len(df)} rows")
df.head()


Created DataFrame with 215353 rows


,ind,endTime,lastUpdatedTime,priceArea,productionGroup,quantityKwh,startTime
0,0,2021-01-01T01:00:00+01:00,2024-12-20T10:35:40+01:00,NO1,hydro,2507716.8,2021-01-01T00:00:00+01:00
1,1,2021-01-01T02:00:00+01:00,2024-12-20T10:35:40+01:00,NO1,hydro,2494728.0,2021-01-01T01:00:00+01:00
2,2,2021-01-01T03:00:00+01:00,2024-12-20T10:35:40+01:00,NO1,hydro,2486777.5,2021-01-01T02:00:00+01:00
3,3,2021-01-01T04:00:00+01:00,2024-12-20T10:35:40+01:00,NO1,hydro,2461176.0,2021-01-01T03:00:00+01:00
4,4,2021-01-01T05:00:00+01:00,2024-12-20T10:35:40+01:00,NO1,hydro,2466969.2,2021-01-01T04:00:00+01:00


In [11]:
# I found that the columns in the Cassandra table was constructed with lowercase letters.
# Therefore, I need to convert the column names to lowercase before writing to Cassandra

name_dict = {}
for capitalname in (df.columns):
    name_dict[capitalname] = capitalname.lower()
name_dict

df = df.rename(columns=name_dict)

# Convert the Pandas DataFrame to Spark DataFrame and save it to Cassandra (append mode)
spark.createDataFrame(df).write.format("org.apache.spark.sql.cassandra")\
.options(table="elhub_api", keyspace="ind320_keyspace").mode("append").save()

### Plotting

In [12]:
spark.read.format("org.apache.spark.sql.cassandra")\
.options(table="elhub_api", keyspace="ind320_keyspace").load()\
.createOrReplaceTempView("elhub_api_view")

df_spark = spark.sql("SELECT priceArea, productionGroup, startTime, quantityKwh FROM elhub_api_view")

In [13]:
area = 'NO1'
df_kwh_byArea = df_spark[df_spark['priceArea'] == area].groupBy('productionGroup').agg({'quantityKwh': 'sum'}).toPandas()

fig = px.pie(df_kwh_byArea, values='sum(quantityKwh)', names='productionGroup', 
             title=f'Total energy production in area {area} by groduction group', 
             color='productionGroup')
fig.show()

In [14]:
area = 'NO1'

df_month = df_spark[(df_spark['priceArea'] == area) & (df_spark['startTime'] < '2021-02-01T00:00:00+01:00')].toPandas()
df_month = df_month.sort_values(by='productionGroup').sort_values(by='startTime').reset_index()
df_month['startTime'] = pd.to_datetime(df_month['startTime'])

fig = px.line(df_month, x='startTime', y='quantityKwh', color='productionGroup')
fig.show()

### Saving df to MongoDB

In [15]:
# Selecting a database and a collection
database = client['project']
collection = database['data']
collection.delete_many({}) # starting fresh

# Convert DataFrame to JSON and dumping to MongoDB
df_pd = df_spark.toPandas()
json_data = df_pd.to_json(orient='records')

documents = json.loads(json_data)
try: 
    collection.insert_many(documents)
    print('Successfully inserted data to MongoDB')
except Exception as e:
    print(e)

Successfully inserted data to MongoDB


### Stopping Sparksession

In [16]:
# Stop Spark session
try:
    spark.stop()
    print('Spark session terminated successfully')
except ConnectionRefusedError:
    print("Spark session already stopped.")
except NameError:
    print('Spark session is not defined')

Spark session terminated successfully


### Testing for Streamlit app

In [17]:
db = client['project']
collection = db['data']

try: 
    items = collection.find()
    items = list(items)
    for i, item in enumerate(items): 
        if i < 10:
            print(item)
except Exception as e: 
    print(e)

{'_id': ObjectId('68fea9bcf09eaf22c1494748'), 'priceArea': 'NO4', 'productionGroup': 'other', 'startTime': '2021-09-20T21:00:00+02:00', 'quantityKwh': 653.7919921875}
{'_id': ObjectId('68fea9bcf09eaf22c1494749'), 'priceArea': 'NO3', 'productionGroup': 'solar', 'startTime': '2021-12-23T14:00:00+01:00', 'quantityKwh': 35.7050018311}
{'_id': ObjectId('68fea9bcf09eaf22c149474a'), 'priceArea': 'NO3', 'productionGroup': 'solar', 'startTime': '2021-12-17T22:00:00+01:00', 'quantityKwh': 83.7279968262}
{'_id': ObjectId('68fea9bcf09eaf22c149474b'), 'priceArea': 'NO2', 'productionGroup': 'hydro', 'startTime': '2021-09-26T09:00:00+02:00', 'quantityKwh': 4899209.5}
{'_id': ObjectId('68fea9bcf09eaf22c149474c'), 'priceArea': 'NO2', 'productionGroup': 'other', 'startTime': '2021-11-01T17:00:00+01:00', 'quantityKwh': 0.3580000103}
{'_id': ObjectId('68fea9bcf09eaf22c149474d'), 'priceArea': 'NO3', 'productionGroup': 'solar', 'startTime': '2021-01-17T21:00:00+01:00', 'quantityKwh': 18.2439994812}
{'_id': 

In [18]:
df = pd.DataFrame(items)
area = 'NO1'

df_kwh_byArea = df[df['priceArea'] == area].groupby('productionGroup').agg({'quantityKwh': 'sum'}).sort_values('quantityKwh').reset_index()

df_kwh_byArea.head()

,productionGroup,quantityKwh
0,other,5.256123e+04
1,solar,1.438194e+07
2,thermal,2.361180e+08
3,wind,5.473603e+08
4,hydro,1.835678e+10


In [19]:
areas = sorted(df["priceArea"].unique().tolist())
areas

['NO1', 'NO2', 'NO3', 'NO4', 'NO5']

In [20]:
prods = sorted(df["productionGroup"].unique().tolist())
prods

['hydro', 'other', 'solar', 'thermal', 'wind']

In [21]:
month_dict = {'January': '2020-01-01',
                'February': '2020-02-01',
                'March': '2020-03-01',
                'April': '2020-04-01', 
                'May': '2020-05-01',
                'June': '2020-06-01',
                'July': '2020-07-01',
                'August': '2020-08-01',
                'September': '2020-09-01',
                'October': '2020-10-01',
                'November': '2020-11-01',
                'December': '2020-12-01'}

month_dict['January']

'2020-01-01'

In [22]:
df.head()

,_id,priceArea,productionGroup,startTime,quantityKwh
0,68fea9bcf09eaf22c1494748,NO4,other,2021-09-20T21:00:00+02:00,6.537920e+02
1,68fea9bcf09eaf22c1494749,NO3,solar,2021-12-23T14:00:00+01:00,3.570500e+01
2,68fea9bcf09eaf22c149474a,NO3,solar,2021-12-17T22:00:00+01:00,8.372800e+01
3,68fea9bcf09eaf22c149474b,NO2,hydro,2021-09-26T09:00:00+02:00,4.899210e+06
4,68fea9bcf09eaf22c149474c,NO2,other,2021-11-01T17:00:00+01:00,3.580000e-01


In [23]:
df_month = df[(df['priceArea'] == area) & (df['startTime'] < '2021-02-01T00:00:00+01:00')]
df_month = df_month.sort_values(by='productionGroup').sort_values(by='startTime').reset_index()
df_month['startTime'] = pd.to_datetime(df_month['startTime'])
df_month

,index,_id,priceArea,productionGroup,startTime,quantityKwh
0,136668,68fea9bdf09eaf22c14b5d24,NO1,other,2021-01-01 00:00:00+01:00,0.000000e+00
1,47114,68fea9bcf09eaf22c149ff52,NO1,thermal,2021-01-01 00:00:00+01:00,5.136904e+04
2,101819,68fea9bdf09eaf22c14ad503,NO1,hydro,2021-01-01 00:00:00+01:00,2.507717e+06
3,102313,68fea9bdf09eaf22c14ad6f1,NO1,wind,2021-01-01 00:00:00+01:00,9.370720e+02
4,179147,68fea9bef09eaf22c14c0313,NO1,solar,2021-01-01 00:00:00+01:00,6.106000e+00
...,...,...,...,...,...,...
3715,17864,68fea9bcf09eaf22c1498d10,NO1,solar,2021-01-31 23:00:00+01:00,9.115000e+00
3716,213136,68fea9bef09eaf22c14c87d8,NO1,hydro,2021-01-31 23:00:00+01:00,2.248686e+06
3717,163610,68fea9bdf09eaf22c14bc662,NO1,other,2021-01-31 23:00:00+01:00,0.000000e+00
3718,25225,68fea9bcf09eaf22c149a9d1,NO1,wind,2021-01-31 23:00:00+01:00,5.050780e+02


In [24]:
df_month["startTime"].dt.month

0       1
1       1
2       1
3       1
4       1
       ..
3715    1
3716    1
3717    1
3718    1
3719    1
Name: startTime, Length: 3720, dtype: int32

In [25]:
df_month['year'] = df_month['startTime'].dt.year
df_month['hour'] = df_month['startTime'].dt.hour

df_month

,index,_id,priceArea,productionGroup,startTime,quantityKwh,year,hour
0,136668,68fea9bdf09eaf22c14b5d24,NO1,other,2021-01-01 00:00:00+01:00,0.000000e+00,2021,0
1,47114,68fea9bcf09eaf22c149ff52,NO1,thermal,2021-01-01 00:00:00+01:00,5.136904e+04,2021,0
2,101819,68fea9bdf09eaf22c14ad503,NO1,hydro,2021-01-01 00:00:00+01:00,2.507717e+06,2021,0
3,102313,68fea9bdf09eaf22c14ad6f1,NO1,wind,2021-01-01 00:00:00+01:00,9.370720e+02,2021,0
4,179147,68fea9bef09eaf22c14c0313,NO1,solar,2021-01-01 00:00:00+01:00,6.106000e+00,2021,0
...,...,...,...,...,...,...,...,...
3715,17864,68fea9bcf09eaf22c1498d10,NO1,solar,2021-01-31 23:00:00+01:00,9.115000e+00,2021,23
3716,213136,68fea9bef09eaf22c14c87d8,NO1,hydro,2021-01-31 23:00:00+01:00,2.248686e+06,2021,23
3717,163610,68fea9bdf09eaf22c14bc662,NO1,other,2021-01-31 23:00:00+01:00,0.000000e+00,2021,23
3718,25225,68fea9bcf09eaf22c149a9d1,NO1,wind,2021-01-31 23:00:00+01:00,5.050780e+02,2021,23
